Burger’s Equation

Burger’s equation in on spatial dimension looks like:

$$\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} = v \frac{\partial^2 u}{\partial x^2}$$

This is combination of linear convection and diffusion. Discretize using the methods we have seen.

$$\frac{u_i^{n+1} - u_i^n}{\Delta t} + u_i^n \frac{u_i^n - u_{i-1}^n}{\Delta x} = v \frac{u_{i+1}^n - 2u_i^n + u_{i-1}^n}{\Delta x^2}$$

We will step in time as follows:

$$u_i^{n+1} = u_i^n - u_i^n\frac{\Delta t}{\Delta x} (u_i^n - u_{i-1}^n) + v \frac{\Delta t}{\Delta x^2}(u_{i+1}^n - 2u_i^n + u_{i-1}^n)$$

Initial and  Boundary Conditions


$$u = -\frac{2v}{\phi} \frac{\partial \phi}{\partial x} + 4$$

$$\phi (t=0) = \phi_0 = exp \left( \frac{-x^2}{4v} \right) + exp \left(  \frac{-(x-2 \pi)^2}{4v} \right ) $$

This has analytical solution:

$$u = -\frac{2v}{\phi}\frac{\partial \phi}{\partial x} + 4$$
$$ \phi = exp \left ( \frac{-(x-4t)^2}{4v(t+1)} \right ) + exp \left ( \frac{-(x-4t-2 \pi)^2}{4v(t+1)} \right )$$

Boundary Condition: 
$$u(0) = u(2 \pi)$$
